In [2]:
import requests
import json
from bs4 import BeautifulSoup
from transformers import pipeline
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import xmltodict, json
import ollama
import pandas as pd


#cik = '000320193' # apple inc CIK

#url = f"https://www.sec.gov/files/company_tickers.json"# to find the company CIK from its ticker (e.g., APPL CIK is 0000320193)
#search_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=8-K&count={count}&output=atom" # to find 8-Ks from CIK

# cik = '000320193' # AAPL

def fetch_cik(ticker):

    driver = webdriver.Chrome()

    # URL to fetch the JSON data
    url = "https://www.sec.gov/files/company_tickers.json"
    cik = None
    # Open the URL
    driver.get(url)

    # Wait for the page to load (you can adjust the time as needed)
    # time.sleep(5)  # Wait for 5 seconds to ensure the page is loaded

    # Get the page source (HTML)
    page_source = driver.find_element(By.TAG_NAME, 'pre').text

    try:
        # Extract JSON data from page source
        json_data = json.loads(page_source)

    except json.JSONDecodeError:
        print("Error decoding the JSON response.")
    for d in json_data:
        if ticker == json_data[d]['ticker']:
            cik = json_data[d]['cik_str']
    # Close the browser
    driver.quit()
    return cik



def fetch_8k_filings(ticker, cik, count):

    driver = webdriver.Chrome()

    # URL to fetch the JSON data
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=8-K&count={count}&output=atom"
    cik = None
    # Open the URL
    driver.get(url)

    # Wait for the page to load (you can adjust the time as needed)
    # time.sleep(5)  # Wait for 5 seconds to ensure the page is loaded

    # Get the page source (HTML)
    page_source = driver.find_element(By.TAG_NAME, 'pre').text
    output_data = []

    try:
        # Extract JSON data from page source
        xml_data = xmltodict.parse(str(page_source))
        json_data = json.loads(json.dumps(xml_data))
        for entry in json_data['feed']['entry']:
            output_data.append(
                {
                    "Company Name": json_data['feed']['company-info']['conformed-name'],
                    "Stock Name": ticker,
                    "Filing Time": entry['content']['filing-date'],
                    "New Product": "",
                    "Product Description": "",
                    "link": entry['content']['filing-href']
                }
            )

    except json.JSONDecodeError:
        print("Error decoding the JSON response.")
    # Close the browser
    driver.quit()
    return output_data



#filings = fetch_8k_filings(ticker, cik, 10)
#filings






   




C:\Users\Robbie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
def get_after_substring(text, substring):
    """
    Returns the part of the string after the first occurrence of a substring.
    If the substring is not found, it returns an empty string.
    """
    index = text.find(substring)
    if index != -1:
        return text[index + len(substring):]
    return ""

In [4]:
tickers = ["AAPL", "TSLA", "VZ", "MSTR", "AXP", "GME", "MSFT", "AMC", "IBM", "GOOGL", "NVDA", "EA"]
# tickers = ["AAPL"]
all_filings = []
options = webdriver.ChromeOptions()
options.add_argument("--enable-javascript")
driver = webdriver.Chrome(options=options)
for ticker in tickers:

    cik = fetch_cik(ticker)

    filings = fetch_8k_filings(ticker, cik, 10)

    for i in range(len(filings)):
       # if i != 0:
          #  continue
        try: 


            url = filings[i]['link']

            print(url)
            driver.get(url)
            

            product_name = driver.find_element(By.XPATH, '//*[@id="contentDiv"]/div[1]/div[2]/div[3]/div[2]').text
            page_source = driver.find_element(By.XPATH, '//*[@id="contentDiv"]/div[2]/div/table/tbody/tr[last()]/td[3]/a').get_attribute('href')
            print(page_source)
            driver.get(page_source)
            #time.sleep(5)
            file = driver.find_element(By.TAG_NAME, 'body').text
            cleaned_file = cleanhtml(file)
            response = ollama.chat(model='deepseek-r1:1.5b', messages=[
                {'role': 'user', 'content': f'summarize this SEC 8-K document for {filings[i]['Company Name']}: {cleaned_file}'},
            ])
           # print(response['message']['content'])
            filings[i]['Product Description'] = get_after_substring(response['message']['content'],"</think>").replace(",","").replace("\n","")[:180]
            filings[i]['New Product'] = product_name
        except:
            continue
    all_filings.extend(filings)
driver.quit()
print(all_filings)

output_csv = []
for i in range(len(all_filings)):
    output_csv.append(
                {
                    "Company Name": all_filings[i]['Company Name'],
                    "Stock Name": all_filings[i]['Stock Name'],
                    "Filing Time": all_filings[i]['Filing Time'],
                    "New Product": all_filings[i]['New Product'],
                    "Product Description": all_filings[i]['Product Description']
                }
            )

df = pd.DataFrame(output_csv)
df.to_csv('filename.csv', index=False)




https://www.sec.gov/Archives/edgar/data/320193/000114036125005876/0001140361-25-005876-index.htm
https://www.sec.gov/Archives/edgar/data/320193/000114036125005876/0001140361-25-005876.txt
https://www.sec.gov/Archives/edgar/data/320193/000032019325000007/0000320193-25-000007-index.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019325000007/0000320193-25-000007.txt
https://www.sec.gov/Archives/edgar/data/320193/000114036125000228/0001140361-25-000228-index.htm
https://www.sec.gov/Archives/edgar/data/320193/000114036125000228/0001140361-25-000228.txt
https://www.sec.gov/Archives/edgar/data/320193/000032019324000120/0000320193-24-000120-index.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019324000120/0000320193-24-000120.txt
https://www.sec.gov/Archives/edgar/data/320193/000114036124040659/0001140361-24-040659-index.htm
https://www.sec.gov/Archives/edgar/data/320193/000114036124040659/0001140361-24-040659.txt
https://www.sec.gov/Archives/edgar/data/320193/0001140361240

In [5]:
df = pd.DataFrame(output_csv)
df.to_csv('filename.csv', index=False)

In [6]:
import ollama
response = ollama.chat(model='deepseek-r1:1.5b', messages=[
        {'role': 'user', 'content': f'just return yes or no: {cleanhtml(file)}'},
    ])
print(response['message']['content'])
response = ollama.chat(model='deepseek-r1:1.5b', messages=[
        {'role': 'user', 'content': f'summarize this SEC 8-K document for Apple in 160 characters: {cleanhtml(file)}'},
    ])

print(response['message']['content'])

<think>
首先，我看到用户提供的 XML 文件包含了关于 XBRL 标准的定义。XBRL（电子艺术出版物）的标准通常用于描述一本书、目录或网页的出版信息。

接下来，我观察到第一行中的“0000712515”应该代表书号，通常为 001 或 003 等，但在中文书籍中使用数字通常是不太常见的。这可能是一个错误或者特殊情况下的编号。

然后，发现日期部分有两个相同的“2024-01-26”，这可能是打印和电子出版物的发布日期相同的情况，通常没有问题。

在公司信息部分，我注意到“ELECTRONIC ARTS INC.”被拼写错误为“EA”。同时，“false”被重复使用多次，这可能是个错误。需要检查是否所有字段都正确填写，特别是是否存在拼写或语法错误。

此外，地址和电话号码看起来没有问题，但日期的格式是否有特殊的要求？通常日期是不需要引号的，所以“2024-01-26”应该是正确的。

最后，确认是否有其他遗漏的信息，比如出版物类型或其他附加信息。根据提供的内容，似乎已经涵盖了所有必要的字段，但存在的错误需要进行修正。
</think>

看起来在你的 XML 文件中存在一些错误：

1. **书号**：`0000712515` 是一个 unusual 的 XBRL 标准编号，通常为 001 或 003。这可能是一个错误。

2. **公司信息**：
   - `ELECTRONIC ARTS INC.` 缺少引号，正确的应是 `"ELECTRONIC ARTS INC."`。
   - 后续字段 `[false]` 出现两次，这可能是个错误。

3. **地址与电话**：看起来没有问题。

如果你需要修正这些问题，请按照以下格式提供：

```
<xbrl
  xmlns: http://www.xbrl.org/2003/instance
  xmlns: dei: http://xbrl.sec.gov/dei/2023
  xmlns:lxml: http://www.w3.org/1999/xlink}
  
    712515
    false
    8-K
    2024-01-26
    ELECTRONIC ARTS INC.
    DE
    0-17948
    94-2838567
    209 Redwood Sh